In [3]:
import os
import math
import jieba
import random
import numpy as np
from sklearn.svm import SVC

In [4]:
cwd = os.getcwd()
jieba.load_userdict('{}/NTUSD.txt'.format(cwd))

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/zz/8hl03_bd6sj_wrd8__z72r8r0000gn/T/jieba.cache
Loading model cost 0.849 seconds.
Prefix dict has been built successfully.


In [5]:
NTUSD_POS = list()
with open('{}/positive.txt'.format(cwd),'r',encoding='utf-8')as file:
    for line in file.readlines():
        line = line.strip()
        NTUSD_POS.append(line)
        
NTUSD_NEG = list()
with open('{}/negative.txt'.format(cwd),'r',encoding='utf-8')as file:
    for line in file.readlines():
        line = line.strip()
        NTUSD_NEG.append(line)
        
NTUSD = list()
with open('{}/NTUSD.txt'.format(cwd),'r',encoding='utf-8')as file:
    for line in file.readlines():
        line = line.strip()
        NTUSD.append(line)

In [6]:
print(NTUSD_POS[5])
print(NTUSD_NEG[102])
print(NTUSD[1200])

了不起
工作过度
神智健全


In [7]:
labels,comments = list(),list()
pos,neg = 0,0
with open('{}/HotelComment_simple.txt'.format(cwd),'r',encoding='utf-8')as file:
    for line in file.readlines():
        temp = [0,0]
        line = line.strip() # line[0] = 0 -> negative; 1 -> positive comment
        temp[int(line[0])]=1 # 0 -> 1 0; 1 -> 0 1
        if temp[0]==1:
            neg+=1
        else:
            pos+=1
        labels.append(temp)
        comments.append(line[2:])
print('pos:{}\nneg:{}'.format(pos,neg))

pos:1000
neg:1000


In [8]:
print(comments[0])
print(labels[0])
print(comments[1000])
print(labels[1000])

标准间太差 房间还不如3星的 而且设施非常陈旧.建议酒店把老的标准间从新改善.
[1, 0]
距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.
[0, 1]


In [9]:
import re

chinese = re.compile(u'[\u4e00-\u9fa5]+')
number = re.compile(u'[\u0030-\u0039]')
english = re.compile(u'[\u0041-\u005a\u0061-\u007a]')

def IsChinese(cks):
    return chinese.search(cks) and not number.search(cks) and not english.search(cks)


In [10]:
PSO = dict()
NSO = dict()
SO = dict()
idf = dict()
tf = list()

for i in range(pos+neg):
    tempTF = dict()
    tempIDF = list()
    for seg in set(jieba.cut(comments[i])):
        if IsChinese(seg):
            #PSO NSO#
            if labels[i]==[1,0]: # 0 -> 1 0 Negtive Comment
                if seg not in NSO:
                    NSO[seg]=2
                else:
                    NSO[seg]+=1
            elif labels[i]==[0,1]: # 1 -> 0 1 Positive Comment
                if seg not in PSO:
                    PSO[seg]=2
                else:
                    PSO[seg]+=1
            #TF IDF#
            tempIDF.append(seg)
            
            if seg not in tempTF:
                tempTF[seg]=1
            else:
                tempTF[seg]+=1
                
    for key in tempTF:
        tempTF[key]=tempTF[key]/len(tempIDF)
        
    tf.append(tempTF)
    
    for c in set(tempIDF):
        if c not in idf:
            idf[c]=1
        else:
            idf[c]+=1

for key in idf:
    idf[key]=math.log(len(comments)/idf[key])
    
for key in PSO:
    PSO[key]/=pos
    
for key in NSO:
    NSO[key]/=neg

words = list(PSO.keys())
words.extend(list(NSO.keys()))
words = set(words)

print(len(words))

for key in words:
    if key in PSO and key in NSO:
        SO[key]=math.log(PSO[key]/NSO[key])
    elif key in PSO and not key in NSO:
        SO[key]=math.log(PSO[key]/(1/neg))
    elif key not in PSO and key in NSO:
        SO[key]=math.log((1/pos)/NSO[key])
# sortKey = sorted(SO,key=lambda x:SO[x],reverse=True)
# for key in sortKey:
#     if key in NTUSD_POS:
#         print('{}+:{}'.format(key,SO[key]))
#     elif key in NTUSD_NEG:
#         print('{}-:{}'.format(key,SO[key]))
#     else:
#         print('{}:{}'.format(key,SO[key]))

12249


In [11]:
TFIDF = list()
for i in range(len(tf)):
    temp = dict()
    for key in tf[i]:
        temp[key] = tf[i][key]*idf[key]
    TFIDF.append(temp)

# for i in range(len(TFIDF)):
#     sortKey = sorted(TFIDF[i],key=lambda x:TFIDF[i][x],reverse=True)
#     for key in sortKey:
#         print('{}:{}'.format(key,TFIDF[i][key]))
#     print('-----------------')

In [12]:
TFSO = list()
for i in range(len(tf)):
    temp = dict()
    for key in tf[i]:
        temp[key] = tf[i][key]*(abs(SO[key])+1)
    TFSO.append(temp)

# for i in range(len(TFSO)):
#     sortKey = sorted(TFSO[i],key=lambda x:TFSO[i][x],reverse=True)
#     for key in sortKey:
#         print('{}:{}'.format(key,TFSO[i][key]))
#     print('-----------------')

In [13]:
TFSOIDF = list()
for i in range(len(TFIDF)):
    temp = dict()
    for key in TFIDF[i]:
        temp[key] = TFIDF[i][key]*(abs(SO[key])+1)
    TFSOIDF.append(temp)

# for i in range(len(TFSOIDF)):
#     sortKey = sorted(TFSOIDF[i],key=lambda x:TFSOIDF[i][x],reverse=True)
#     for key in sortKey:
#         print('{}:{}'.format(key,TFSOIDF[i][key]))
#     print('-----------------')

In [14]:
TFSDIDF = list()
k = 2
for i in range(len(TFIDF)):
    temp = dict()
    for key in TFIDF[i]:
        if key in NTUSD:
            temp[key] = TFIDF[i][key]*k
        else:
            temp[key] = TFIDF[i][key]
    TFSDIDF.append(temp)

# for i in range(len(TFSDIDF)):
#     sortKey = sorted(TFSDIDF[i],key=lambda x:TFSDIDF[i][x],reverse=True)
#     for key in sortKey:
#         print('{}:{}'.format(key,TFSDIDF[i][key]))
#     print('-----------------')

In [15]:
TFSSIDF = list()
k = 2
for i in range(len(TFSOIDF)):
    temp = dict()
    for key in TFSOIDF[i]:
        if key in NTUSD:
            temp[key] = TFSOIDF[i][key]*k
        else:
            temp[key] = TFSOIDF[i][key]
    TFSSIDF.append(temp)

# for i in range(len(TFSSIDF)):
#     sortKey = sorted(TFSSIDF[i],key=lambda x:TFSSIDF[i][x],reverse=True)
#     for key in sortKey:
#         print('{}:{}'.format(key,TFSSIDF[i][key]))
#     print('-----------------')

In [16]:
def trainSVM(swt,lab,words):
    x,y = list(),list()
    for i in range(len(swt)):
        temp = list()
        for word in words:
            if word in swt[i]:
                temp.append(swt[i][word])
            else:
                temp.append(0)
        x.append(temp)
        if lab[i]==[0,1]:
            y.append(1)
        elif lab[i]==[1,0]:
            y.append(0)
    x,y = np.array(x),np.array(y)
    clf = SVC(kernel='linear', C=1.0)
    clf.fit(x,y)
    return clf

In [17]:
def testSVM(clf,swt,lab,words):
    x,y = list(),list()
    for i in range(len(swt)):
        temp = list()
        for word in words:
            if word in swt[i]:
                temp.append(swt[i][word])
            else:
                temp.append(0)
        x.append(temp)
        if lab[i]==[0,1]:
            y.append(1)
        elif lab[i]==[1,0]:
            y.append(0)
    x,y = np.array(x),np.array(y)
    predict=clf.predict(x)
    correct,wrong = 0,0
    for i in range(len(predict)):
        if y[i]==predict[i]:
            correct+=1
        else:
            wrong+=1
    print(round(correct/(correct+wrong),2))

In [18]:
index = [i for i in range(len(labels))] # index = [0, 1, 2, ..., 1999]
trainIndex = random.sample(index, int(len(labels)*0.8))
testIndex = list()
for i in index:
    if i not in trainIndex:
        testIndex.append(i)

# trainIndex = index
# testIndex = trainIndex
print('train:{}'.format(len(trainIndex)))
print('test:{}'.format(len(testIndex)))

print('TFIDF_SVM Accuracy:')
TFIDF_SVM = trainSVM([TFIDF[i] for i in trainIndex],[labels[i] for i in trainIndex],words)
testSVM(TFIDF_SVM,[TFIDF[i] for i in testIndex],[labels[i] for i in testIndex],words)

print('TFSO_SVM Accuracy:')
TFSO_SVM = trainSVM([TFSO[i] for i in trainIndex],[labels[i] for i in trainIndex],words)
testSVM(TFSO_SVM,[TFSO[i] for i in testIndex],[labels[i] for i in testIndex],words)

print('TFSOIDF_SVM Accuracy:')
TFSOIDF_SVM = trainSVM([TFSOIDF[i] for i in trainIndex],[labels[i] for i in trainIndex],words)
testSVM(TFSOIDF_SVM,[TFSOIDF[i] for i in testIndex],[labels[i] for i in testIndex],words)

print('TFSDIDF_SVM Accuracy:')
TFSDIDF_SVM = trainSVM([TFSDIDF[i] for i in trainIndex],[labels[i] for i in trainIndex],words)
testSVM(TFSDIDF_SVM,[TFSDIDF[i] for i in testIndex],[labels[i] for i in testIndex],words)

print('TFSSIDF_SVM Accuracy:')
TFSSIDF_SVM = trainSVM([TFSSIDF[i] for i in trainIndex],[labels[i] for i in trainIndex],words)
testSVM(TFSSIDF_SVM,[TFSSIDF[i] for i in testIndex],[labels[i] for i in testIndex],words)

train:1600
test:400
TFIDF_SVM Accuracy:
0.88
TFSO_SVM Accuracy:
0.81
TFSOIDF_SVM Accuracy:
0.94
TFSDIDF_SVM Accuracy:
0.89
TFSSIDF_SVM Accuracy:
0.93


In [27]:
index = [i for i in range(len(labels))]
trainIndex = index
testIndex = trainIndex
print('train:{}'.format(len(trainIndex)))
print('test:{}'.format(len(testIndex)))

print('TFIDF_SVM Accuracy:')
TFIDF_SVM = trainSVM([TFIDF[i] for i in trainIndex],[labels[i] for i in trainIndex],words)
testSVM(TFIDF_SVM,[TFIDF[i] for i in testIndex],[labels[i] for i in testIndex],words)

print('TFSO_SVM Accuracy:')
TFSO_SVM = trainSVM([TFSO[i] for i in trainIndex],[labels[i] for i in trainIndex],words)
testSVM(TFSO_SVM,[TFSO[i] for i in testIndex],[labels[i] for i in testIndex],words)

print('TFSOIDF_SVM Accuracy:')
TFSOIDF_SVM = trainSVM([TFSOIDF[i] for i in trainIndex],[labels[i] for i in trainIndex],words)
testSVM(TFSOIDF_SVM,[TFSOIDF[i] for i in testIndex],[labels[i] for i in testIndex],words)

print('TFSDIDF_SVM Accuracy:')
TFSDIDF_SVM = trainSVM([TFSDIDF[i] for i in trainIndex],[labels[i] for i in trainIndex],words)
testSVM(TFSDIDF_SVM,[TFSDIDF[i] for i in testIndex],[labels[i] for i in testIndex],words)

print('TFSSIDF_SVM Accuracy:')
TFSSIDF_SVM = trainSVM([TFSSIDF[i] for i in trainIndex],[labels[i] for i in trainIndex],words)
testSVM(TFSSIDF_SVM,[TFSSIDF[i] for i in testIndex],[labels[i] for i in testIndex],words)

train:2000
test:2000
TFIDF_SVM Accuracy:
0.96
TFSO_SVM Accuracy:
0.85
TFSOIDF_SVM Accuracy:
0.99
TFSDIDF_SVM Accuracy:
0.97
TFSSIDF_SVM Accuracy:
0.99
